# SciPy Signal Processing

FFTs  analysis, filtering, convolution and correlation, peak extraction, and spectral summaries with practical examples.

In [1]:
import numpy as np
from scipy import signal

np.set_printoptions(precision=4, suppress=True)

## 1. Fourier Transform and Spectral Analysis
FFT transforms signals to frequency space so dominant periodic components can be identified.

In [2]:
rng = np.random.default_rng(30)
fs = 200
t = np.arange(0, 5, 1 / fs)

vibration = 0.9 * np.sin(2 * np.pi * 18 * t) + 0.45 * np.sin(2 * np.pi * 45 * t) + 0.2 * rng.normal(size=t.size)
freqs = np.fft.rfftfreq(t.size, d=1 / fs)
amplitudes = np.abs(np.fft.rfft(vibration))
main_freq = freqs[np.argmax(amplitudes[1:]) + 1]

print("Example 1: Machine vibration diagnostics")
print(f"Dominant frequency: {main_freq:.2f} Hz")

power_signal = 1.1 * np.sin(2 * np.pi * 60 * t) + 0.25 * np.sin(2 * np.pi * 180 * t) + 0.1 * rng.normal(size=t.size)
power_fft = np.abs(np.fft.rfft(power_signal))

print()
print("Example 2: Power quality monitoring")
print(f"Amplitude near 60 Hz: {power_fft[np.argmin(np.abs(freqs - 60))]:.3f}")
print(f"Amplitude near 180 Hz: {power_fft[np.argmin(np.abs(freqs - 180))]:.3f}")

Example 1: Machine vibration diagnostics
Dominant frequency: 18.00 Hz

Example 2: Power quality monitoring
Amplitude near 60 Hz: 550.770
Amplitude near 180 Hz: 4.697


## 2. Digital Filters
Filters isolate useful signal bands and suppress unwanted noise.

In [3]:
rng = np.random.default_rng(14)
fs = 100
t = np.arange(0, 20, 1 / fs)

heart_rate_signal = 72 + 4 * np.sin(2 * np.pi * 1.2 * t) + 1.8 * np.sin(2 * np.pi * 12 * t)
heart_rate_noisy = heart_rate_signal + rng.normal(0, 1.3, size=t.size)
b_lp, a_lp = signal.butter(4, 3 / (fs / 2), btype='low')
heart_rate_clean = signal.filtfilt(b_lp, a_lp, heart_rate_noisy)

print("Example 1: Wearable heart-rate denoising")
print(f"Noisy std: {np.std(heart_rate_noisy):.3f}")
print(f"Filtered std: {np.std(heart_rate_clean):.3f}")

seismic = 0.2 * np.sin(2 * np.pi * 0.8 * t) + 1.2 * np.sin(2 * np.pi * 6 * t) + 0.2 * np.sin(2 * np.pi * 20 * t)
b_bp, a_bp = signal.butter(4, [4 / (fs / 2), 10 / (fs / 2)], btype='bandpass')
seismic_band = signal.filtfilt(b_bp, a_bp, seismic)

print()
print("Example 2: Seismic band isolation")
print(f"Original RMS: {np.sqrt(np.mean(seismic ** 2)):.3f}")
print(f"Band-pass RMS: {np.sqrt(np.mean(seismic_band ** 2)):.3f}")

Example 1: Wearable heart-rate denoising
Noisy std: 3.342
Filtered std: 2.860

Example 2: Seismic band isolation
Original RMS: 0.872
Band-pass RMS: 0.848


## 3. Convolution and Correlation
Convolution smooths and models responses. Correlation detects known patterns in noisy streams.

In [4]:
rng = np.random.default_rng(22)
demand = 200 + 0.8 * np.arange(60) + rng.normal(0, 8, size=60)
smoothed = signal.convolve(demand, np.ones(7) / 7, mode='same')

print("Example 1: Weekly demand smoothing")
print(f"Raw variance: {np.var(demand):.3f}")
print(f"Smoothed variance: {np.var(smoothed):.3f}")

sensor = rng.normal(0, 0.6, size=500)
pulse = np.array([0.2, 1.0, 2.0, 1.0, 0.2])
insert_idx = 280
sensor[insert_idx:insert_idx + pulse.size] += pulse
corr = signal.correlate(sensor, pulse, mode='same')
detected_idx = np.argmax(corr)

print()
print("Example 2: Pulse detection with matched filtering")
print(f"Inserted index: {insert_idx}")
print(f"Detected index: {detected_idx}")

Example 1: Weekly demand smoothing
Raw variance: 323.744
Smoothed variance: 712.319

Example 2: Pulse detection with matched filtering
Inserted index: 280
Detected index: 282


## 4. Peak Finding and Feature Extraction
Peak detection extracts event timing and rate features from temporal signals.

In [5]:
rng = np.random.default_rng(19)
fs = 250
t = np.arange(0, 12, 1 / fs)

ecg_like = 0.08 * np.sin(2 * np.pi * 0.5 * t)
for beat in np.arange(0.8, 12, 1.0):
    ecg_like += 1.2 * np.exp(-((t - beat) ** 2) / (2 * 0.012 ** 2))
ecg_like += rng.normal(0, 0.03, size=t.size)
peaks_ecg, _ = signal.find_peaks(ecg_like, distance=fs * 0.6, prominence=0.4)

print("Example 1: ECG R-peak counting")
print(f"Detected beats: {len(peaks_ecg)}")
print(f"Estimated BPM: {len(peaks_ecg) / (t[-1] / 60):.2f}")

traffic = 180 + 35 * np.sin(2 * np.pi * np.arange(0, 144) / 24) + rng.normal(0, 7, size=144)
traffic[[40, 88, 120]] += [45, 55, 60]
peaks_traffic, props = signal.find_peaks(traffic, prominence=20)

print()
print("Example 2: Website traffic surge detection")
print(f"Peak hours: {peaks_traffic.tolist()}")
print(f"Peak prominences: {np.round(props['prominences'], 2).tolist()}")

Example 1: ECG R-peak counting
Detected beats: 12
Estimated BPM: 60.02

Example 2: Website traffic surge detection
Peak hours: [4, 8, 28, 40, 51, 55, 78, 88, 103, 120, 126]
Peak prominences: [21.6, 49.12, 70.95, 49.52, 29.54, 87.46, 75.48, 46.27, 82.65, 105.65, 34.53]


## 5. PSD and Spectrogram
Power spectral density and spectrograms summarize frequency content globally and over time.

In [6]:
rng = np.random.default_rng(33)
fs = 256
t = np.arange(0, 8, 1 / fs)

machine_noise = 0.7 * np.sin(2 * np.pi * 35 * t) + 0.4 * np.sin(2 * np.pi * 90 * t) + 0.15 * rng.normal(size=t.size)
f_welch, pxx = signal.welch(machine_noise, fs=fs, nperseg=512)

print("Example 1: PSD for rotating equipment")
print(f"Peak PSD frequency: {f_welch[np.argmax(pxx)]:.2f} Hz")

chirp_signal = signal.chirp(t, f0=8, t1=t[-1], f1=110, method='linear') + 0.05 * rng.normal(size=t.size)
f_spec, t_spec, sxx = signal.spectrogram(chirp_signal, fs=fs, nperseg=128, noverlap=96)
strong_bins = np.argmax(sxx, axis=0)

print()
print("Example 2: Frequency sweep tracking")
print(f"Start dominant frequency: {f_spec[strong_bins[0]]:.2f} Hz")
print(f"End dominant frequency: {f_spec[strong_bins[-1]]:.2f} Hz")

Example 1: PSD for rotating equipment
Peak PSD frequency: 35.00 Hz

Example 2: Frequency sweep tracking
Start dominant frequency: 12.00 Hz
End dominant frequency: 106.00 Hz
